In [1]:
# https://www.kaggle.com/amikhshibu/shape-recognition-with-inceptionv3-acc-95
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report

2022-02-01 13:26:47.449621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-01 13:26:47.449664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
 # Split dataset into Train, Validation & Test sets

parent_folder = './train_data'
size = 299   # Resize all images to (size,size)
bs = 32      # Batch size


# Data augmentation on train dataset only
train_data_gen = ImageDataGenerator(width_shift_range = 0.1, height_shift_range = 0.1, zoom_range=0.1, shear_range=0.1, brightness_range=[0.8,1.2], validation_split=0.15, preprocessing_function=preprocess_input)
train_data = train_data_gen.flow_from_directory(parent_folder, class_mode='categorical', target_size=(size,size), color_mode='rgb', batch_size=bs, seed=42, subset='training')

validation_data_gen = ImageDataGenerator(validation_split=0.15, preprocessing_function=preprocess_input)
validation_data = validation_data_gen.flow_from_directory(parent_folder, class_mode='categorical', target_size=(size,size), color_mode='rgb', batch_size=bs, seed=42, subset='validation')

test_data_gen = ImageDataGenerator(validation_split=0.10, preprocessing_function=preprocess_input)
test_data = test_data_gen.flow_from_directory(parent_folder, class_mode='categorical', target_size=(size,size), color_mode='rgb', subset='validation', shuffle=False)

Found 357 images belonging to 4 classes.
Found 63 images belonging to 4 classes.
Found 42 images belonging to 4 classes.


In [3]:
# Assign essential variables
shape = train_data.image_shape                 # Shape of train images (height,width,channels)
print(shape)
k = train_data.num_classes                     # Total number of labels or classes
train_samples = train_data.samples             # Total number of images in train set
validation_samples = validation_data.samples   # total number of images in validation set

(299, 299, 3)


In [4]:
# Build the model

input = Input(shape=shape)
 
basemodel = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=shape, pooling='avg')   # Basemodel is InceptionV3 with pretrained weights trained on imagenet dataset
basemodel.trainable = False                                                                        # Freeze the weights in all layers of the CNN
 
x = basemodel(input)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(k, activation='softmax')(x)
 
model = Model(input,output)

219070464/219055592 [==============================] - 4s 0us/step


In [5]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Initialize callbacks
 
stop = EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)                                             # Stops training early to prevent overfitting
checkpoint = ModelCheckpoint(filepath='./weights/{val_loss:.4f}-weights-{epoch:02d}.hdf5', monitor='val_loss', mode='min', save_best_only=True)   # Saves the model for every best val_loss

In [7]:
# Train the model

ep = 50                      # Number of epochs
spe = train_samples/bs       # Steps per epoch
vs = validation_samples/bs   # Validation steps
 
r = model.fit(train_data, validation_data=validation_data, steps_per_epoch=spe, validation_steps=vs, epochs=ep, callbacks=[stop,checkpoint])

Epoch 1/50
11/11 [==============================] - 33s 2s/step - loss: 0.4205 - accuracy: 0.8207 - val_loss: 0.1033 - val_accuracy: 0.9524
Epoch 2/50
11/11 [==============================] - 25s 2s/step - loss: 0.0437 - accuracy: 0.9860 - val_loss: 0.0112 - val_accuracy: 1.0000
Epoch 3/50
11/11 [==============================] - 30s 3s/step - loss: 0.0164 - accuracy: 0.9972 - val_loss: 0.0138 - val_accuracy: 1.0000
Epoch 4/50
11/11 [==============================] - 35s 3s/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 5/50
11/11 [==============================] - 27s 2s/step - loss: 0.0163 - accuracy: 0.9944 - val_loss: 0.0119 - val_accuracy: 1.0000
Epoch 6/50
11/11 [==============================] - 27s 2s/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0118 - val_accuracy: 1.0000
Epoch 7/50
11/11 [==============================] - 27s 2s/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 8/50
11/11 [==

In [8]:
# Predictions on the test data

pred = model.predict(test_data).argmax(axis=1)
labels = list(train_data.class_indices.keys())

In [9]:
# Get the F1 score on test data prediction

print(classification_report(test_data.classes, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00        10

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42



In [10]:
model.save('model/model_classification_20220116')

INFO:tensorflow:Assets written to: model/model_classification_20220116\assets


In [13]:
img_path = 'triangle.png'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img) 
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x).argmax(axis=1)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', preds)
# print(classification_report(test_data.classes, pred))

Predicted: [3]


In [ ]:
model.summary()

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio

In [13]:
batch_size = 64
num_channels = 1
num_classes = 10
image_size = 28
latent_dim = 128

In [14]:
# We'll use all the available examples from both the training and test
# sets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])

# Scale the pixel values to [0, 1] range, add a channel dimension to
# the images, and one-hot encode the labels.
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
all_labels = keras.utils.to_categorical(all_labels, 10)

# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

11501568/11490434 [==============================] - 2s 0us/step
Shape of training images: (70000, 28, 28, 1)
Shape of training labels: (70000, 10)


In [15]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

138 11


In [16]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

# Image Converters

In [49]:
from tensorflow.keras.preprocessing import image as KerasImage
from PIL import Image
import io
import numpy as np
import base64

In [65]:
def resize_image_from_bytes(pre_image, original_size: tuple, target_size: tuple):
    img = np.frombuffer(pre_image, dtype=np.byte).reshape(original_size + (3,))
    img = KerasImage.array_to_img(img)
    return img.resize(target_size)

def resize_image_from_png(pre_image, target_size: tuple):
    img = Image.open(io.BytesIO(pre_image))
    return img.resize(target_size)

def load_image(filename: str):
    img = KerasImage.load_img(filename, target_size=(299, 299))
    arr_img = KerasImage.img_to_array(img)
    return arr_img


In [39]:
#
circle_rgb = load_image('test_data/circle.png')
dtype = np.dtype('B')
circle_png = None
with open('test_data/circle.png', 'rb') as f:
    circle_png = np.fromfile(f, np.byte)

In [55]:
img = resize_image_from_png(circle_png, (299, 299))

In [75]:
if img.size == (299, 299):
    print('true')
else:
    print('false')

true


In [60]:
from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [57]:
model = keras.models.load_model('model/model_classification.model')

In [69]:
#model.predict(preprocess_input(np.expand_dims(img, axis=0)))
model.predict(preprocess_input(np.expand_dims(load_image('test_data/circle.png'), axis=0)))

TypeError: array() got an unexpected keyword argument 'axis'

# GAN

In [58]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [59]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=20)

NameError: name 'discriminator' is not defined

In [ ]:
# We first extract the trained generator from our Conditiona GAN.
trained_gen = cond_gan.generator

# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_interpolation = 9  # @param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = tf.cast(first_label, tf.float32)
    second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 1  # @param {type:"slider", min:0, max:9, step:1}
end_class = 5  # @param {type:"slider", min:0, max:9, step:1}

fake_images = interpolate_class(start_class, end_class)

In [ ]:
fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
converted_images = tf.image.resize(converted_images, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation.gif", converted_images, fps=1)
embed.embed_file("animation.gif")

In [1]:
import os

# Rename Images

In [10]:
i = 0
for file in os.listdir('./train_data/0_circle/'):
    os.rename('./train_data/0_circle/' + file, f"./train_data/0_circle/tmp_circle_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/0_circle/'):
    os.rename(f"./train_data/0_circle/tmp_circle_{i:04}.png", f"./train_data/0_circle/circle_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/1_square/'):
    os.rename('./train_data/1_square/' + file, f"./train_data/1_square/tmp_square_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/1_square/'):
    os.rename(f"./train_data/1_square/tmp_square_{i:04}.png", f"./train_data/1_square/square_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/2_star/'):
    os.rename('./train_data/2_star/' + file, f"./train_data/2_star/tmp_star_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/2_star/'):
    os.rename(f"./train_data/2_star/tmp_star_{i:04}.png", f"./train_data/2_star/star_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/3_triangle/'):
    os.rename('./train_data/3_triangle/' + file, f"./train_data/3_triangle/tmp_triangle_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/3_triangle/'):
    os.rename(f"./train_data/3_triangle/tmp_triangle_{i:04}.png", f"./train_data/3_triangle/triangle_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/4_lightning/'):
    os.rename('./train_data/4_lightning/' + file, f"./train_data/4_lightning/tmp_lightning_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/4_lightning/'):
    os.rename(f"./train_data/4_lightning/tmp_lightning_{i:04}.png", f"./train_data/4_lightning/lightning_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/5_hourglass/'):
    os.rename('./train_data/5_hourglass/' + file, f"./train_data/5_hourglass/tmp_hourglass_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/5_hourglass/'):
    os.rename(f"./train_data/5_hourglass/tmp_hourglass_{i:04}.png", f"./train_data/5_hourglass/hourglass_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/6_infinity/'):
    os.rename('./train_data/6_infinity/' + file, f"./train_data/6_infinity/tmp_infinity_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/6_infinity/'):
    os.rename(f"./train_data/6_infinity/tmp_infinity_{i:04}.png", f"./train_data/6_infinity/infinity_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/7_swirl/'):
    os.rename('./train_data/7_swirl/' + file, f"./train_data/7_swirl/tmp_swirl_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/7_swirl/'):
    os.rename(f"./train_data/7_swirl/tmp_swirl_{i:04}.png", f"./train_data/7_swirl/swirl_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/8_fire/'):
    os.rename('./train_data/8_fire/' + file, f"./train_data/8_fire/tmp_fire_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/8_fire/'):
    os.rename(f"./train_data/8_fire/tmp_fire_{i:04}.png", f"./train_data/8_fire/fire_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/9_water/'):
    os.rename('./train_data/9_water/' + file, f"./train_data/9_water/tmp_water_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/9_water/'):
    os.rename(f"./train_data/9_water/tmp_water_{i:04}.png", f"./train_data/9_water/water_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/10_arrow/'):
    os.rename('./train_data/10_arrow/' + file, f"./train_data/10_arrow/tmp_arrow_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/10_arrow/'):
    os.rename(f"./train_data/10_arrow/tmp_arrow_{i:04}.png", f"./train_data/10_arrow/arrow_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/11_left/'):
    os.rename('./train_data/11_left/' + file, f"./train_data/11_left/tmp_left_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/11_left/'):
    os.rename(f"./train_data/11_left/tmp_left_{i:04}.png", f"./train_data/11_left/left_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/12_right/'):
    os.rename('./train_data/12_right/' + file, f"./train_data/12_right/tmp_right_{i:04}.png")
    i += 1
i = 0
for file in os.listdir('./train_data/12_right/'):
    os.rename(f"./train_data/12_right/tmp_right_{i:04}.png", f"./train_data/12_right/right_{i:04}.png")
    i += 1